loading data from kaggle to colab

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
 !mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
 ! kaggle competitions download -c ngwl-predict-customer-churn --force

imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import itertools
import gc
from datetime import datetime, timedelta, date

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
addresses = pd.read_csv('addresses.csv.zip')
addresses.shape

(2766724, 2)

In [ ]:
ship1 = pd.read_csv('shipments2020-01-01.csv.zip')
ship2 = pd.read_csv('shipments2020-03-01.csv.zip')
ship3 = pd.read_csv('shipments2020-04-30.csv.zip')
ship4 = pd.read_csv('shipments2020-06-29.csv.zip')
ship1.shape, ship2.shape, ship3.shape, ship4.shape

((141546, 20), (400963, 20), (845419, 20), (872758, 20))

In [ ]:
#concatenate all shipments
all_shipments = pd.concat([ship1, ship2, ship3, ship4])
del ship1, ship2, ship3, ship4
gc.collect()

77

In [ ]:
all_shipments.shape

(2260686, 20)

In [ ]:
#get phone_id from addresses
all_shipments = all_shipments.merge(addresses, left_on='ship_address_id', right_on='id', how='left').drop(['id'], axis=1).drop_duplicates()
all_shipments.shape

(2260686, 21)

In [ ]:
#get calendar month from order completion timestamp
all_shipments['month'] = pd.to_datetime(all_shipments.order_completed_at).dt.month

### features: nr of cancelled/completed orders

In [ ]:
features = []
#aggregate features for each month (April-Sep)
for month in range(4, 10):
  #take 3 months history
  temp = all_shipments[(all_shipments.month<month)&(all_shipments.month>=month-3)]
  #leave only cancelled/complete states
  temp = temp[temp['s.order_state'].isin(['complete', 'canceled'])]
  #get nr of cancelled/completed orders for each customer at each month
  f = temp.pivot_table(index=['phone_id'], columns=['month', 's.order_state'], aggfunc='size', fill_value=0)
  #rename columns
  f.columns = ['canc_1', 'comp_1', 'canc_2', 'comp_2', 'canc_3', 'comp_3']
  #change indices
  f.index = f.index.astype(str)+'_2020-0'+str(month)
  features.append(f)
features_all = pd.concat(features)

In [ ]:
#save features
features_all.to_pickle('/content/gdrive/My Drive/cancelled_completed_features.pkl')

### features: statistics from shipments

In [ ]:
features = []
for month in range(4, 10):
  temp = all_shipments[(all_shipments.month<month)&(all_shipments.month>=month-3)]
  f = temp.groupby(['phone_id', 'month']).agg({'retailer':['nunique'], 'total_cost':[np.mean, 'max', 'min'], 
                         'total_weight':[np.mean, 'max', 'min'], 'rate': [np.count_nonzero, 'sum', 'max']}).unstack()
  stat1, stat2 = ['mean', 'max', 'min'], ['count_not_zero', 'sum', 'max']
  cols = ['ret_nr']+['cost_'+stat for stat in stat1]+['weight_'+stat for stat in stat1]+['rate_'+stat for stat in stat2]  
  months = [1, 2, 3]
  f.columns = [pair[0]+'_'+str(pair[1]) for pair in itertools.product(cols, months)]
  f.index = f.index.astype(str)+'_2020-0'+str(month)
  features.append(f)
features_all = pd.concat(features).fillna(-1)

In [ ]:
#save data
features_all.to_pickle('/content/gdrive/My Drive/retailer_other_stats_features.pkl')

### features: statistics on delivery time

In [ ]:
#get shipment duration in hours
all_shipments['duration'] = (pd.to_datetime(all_shipments.shipped_at, format='%Y-%m-%d %H:%M:%S') - pd.to_datetime(all_shipments.shipment_starts_at, format='%Y-%m-%d %H:%M:%S')).astype('timedelta64[h]')

In [ ]:
lb = all_shipments.duration.quantile(0.01)
ub = all_shipments.duration.quantile(0.99)
lb, ub

(-1.0, 4.0)

In [ ]:
#change outliers to nan
all_shipments['duration'] = np.where((all_shipments['duration']<lb)|(all_shipments['duration']>ub), np.nan, all_shipments['duration'])

In [ ]:
features = []
for month in range(4, 10):
  temp = all_shipments[(all_shipments.month<month)&(all_shipments.month>=month-3)]
  f = temp.groupby(['phone_id', 'month']).agg({'duration':[np.mean, 'max', 'min']}).unstack()
  stats = ['mean', 'max', 'min']
  cols = ['duration_'+stat for stat in stats]
  months = [1, 2, 3]
  f.columns = [pair[0]+'_'+str(pair[1]) for pair in itertools.product(cols, months)]
  f.index = f.index.astype(str)+'_2020-0'+str(month)
  features.append(f)
features_all = pd.concat(features).fillna(-1)

In [ ]:
#save data
features_all.to_pickle('/content/gdrive/My Drive/duration_features.pkl')

### features: nr of messages received

In [ ]:
messages = pd.read_csv('messages.csv.zip')
messages.shape

(18275907, 3)

In [ ]:
#change timestamp to date
messages.sent = pd.to_datetime(messages.sent,unit='s')
#get month
messages['month'] = messages.sent.dt.month

In [ ]:
#get nr of messages per month
agg_messages = messages.groupby(['user_id', 'month']).sent.count().reset_index()

In [ ]:
#get phone_id from shipments
agg_messages = agg_messages.merge(all_shipments[['user_id', 'phone_id']], left_on='user_id', right_on='user_id').drop_duplicates()

In [ ]:
features = []
for month in range(4, 10):
  temp = agg_messages[(agg_messages.month<month)&(agg_messages.month>=month-3)]
  f = temp.groupby(['phone_id', 'month']).sent.sum().unstack()
  months = [1, 2, 3]
  f.columns = ['messages_sent'+'_'+str(m) for m in months]
  f.index = f.index.astype(str)+'_2020-0'+str(month)
  features.append(f)
features_all = pd.concat(features).fillna(0)

In [ ]:
#save data
features_all.to_pickle('/content/gdrive/My Drive/messages_sent.pkl')

#### features: nr of messages received per type

In [ ]:
actions = pd.read_csv('actions.csv')
actions.shape

(962, 4)

In [ ]:
#get action type
messages = messages.merge(actions[['id', 'type']], left_on='action_id', right_on='id', how='left').drop(['id'], axis=1)
messages.shape

(18275907, 5)

In [ ]:
agg_messages_type = messages.groupby(['user_id', 'month', 'type']).sent.count().reset_index()

In [ ]:
agg_messages_type = agg_messages_type.merge(all_shipments[['user_id', 'phone_id']], left_on='user_id', right_on='user_id').drop_duplicates()

In [ ]:
agg_messages_type.type.value_counts()

push     1789688
email     968766
sms       149988
Name: type, dtype: int64

In [ ]:
#sms were started only in August
agg_messages_type[agg_messages_type.type=='sms'].month.value_counts()

8    149988
Name: month, dtype: int64

In [ ]:
#changed message types to push/other
agg_messages_type.loc[agg_messages_type.type!='push', 'type'] = 'other'

In [ ]:
agg_messages_type.type.value_counts()

push     1789688
other    1118754
Name: type, dtype: int64

In [ ]:
features = []
for month in range(4, 10):
  temp = agg_messages_type[(agg_messages_type.month<month)&(agg_messages_type.month>=month-3)]
  f = temp.groupby(['phone_id', 'month', 'type']).sent.sum().unstack().unstack()
  months = [1, 2, 3]
  types = ['other', 'push']
  f.columns = [pair[0]+'_'+str(pair[1]) for pair in itertools.product(types, months)]
  f.index = f.index.astype(str)+'_2020-0'+str(month)
  print(month, f.shape)
  features.append(f)
features_all = pd.concat(features).fillna(0)

4 (148620, 6)
5 (243744, 6)
6 (346423, 6)
7 (411912, 6)
8 (461577, 6)
9 (495648, 6)


In [ ]:
features_all.shape

(2107924, 6)

In [ ]:
#save data
features_all.to_csv('/content/gdrive/My Drive/messages_sent_by_type.csv')

In [ ]:
#save data
features_all.to_pickle('/content/gdrive/My Drive/messages_sent_by_type.pkl')

### user profile features

In [ ]:
users = pd.read_csv('user_profiles.csv.zip')
users.shape

(396820, 3)

In [ ]:
#add phone_id
users = users.merge(all_shipments[['user_id', 'phone_id']], left_on='user_id', right_on='user_id').drop_duplicates()
users.shape

(85073, 4)

In [ ]:
#extract city from shipments
city = all_shipments.groupby('phone_id')['s.city_name'].apply(lambda x:x.value_counts().index[0])


phone_id
2       Москва
3       Москва
4       Москва
5    Челябинск
6       Москва
Name: s.city_name, dtype: object

In [ ]:
users = users.merge(city, left_on='phone_id', right_index=True)
users.drop(['user_id'], axis=1, inplace=True)

In [ ]:
#change birthdate to age
def get_age(bdate):
    today = date.today()
    return today.year - bdate.year - ((today.month, today.day) < (bdate.month, bdate.day))
users['bdate'] = pd.to_datetime(users['bdate'], errors='coerce')
users['age'] = users.bdate.apply(get_age)

In [ ]:
users.drop(['bdate'], axis=1, inplace=True)

In [ ]:
users.rename(columns={'s.city_name':'city'}, inplace=True)

In [ ]:
#save data
users.to_pickle('/content/gdrive/My Drive/user_features.pkl')

In [ ]:
users.to_csv('/content/gdrive/My Drive/user_features.csv')